### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [5]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [6]:
importlib.reload(train)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
# ]
] + train.Metrics.STATISTICAL

In [7]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)


random.seed(74)

model_phases = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

threshold = "fixed"

anxiety_label_type = None
x, y = train.Train_APD.get_apd_data_ranking(
    metrics, model_phases, verbose=False, anxiety_label_type=anxiety_label_type, 
    threshold=threshold, normalize=True, combine_phases=False, standardize=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

# drop subjects with noisy data
# x = x[~x["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y = y[~y["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x.drop(labels=["anxietyGroup"], axis=1)
    
acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    # features = {name: metrics for name in models.keys()}
    features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.62561187 0.63304422 0.62665    0.6347437  0.63474028 0.61534931
 0.63112093 0.63723654 0.62662994 0.63457461 0.61718166 0.6216823
 0.62571921 0.62176901 0.62754308 0.59891376 0.62537858 0.62892599
 0.62827158 0.6339188  0.59819141 0.61827247 0.62974246 0.63525012
 0.63689941        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [49 18], Predictions: [0 1], [64  3]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [49 18], Predictions: [0 1], [52 15]
Training RF ...
Model RF, Actual: [0 1], [49 18], Predictions: [0 1], [43 24]
Training XGB ...
Model XGB, Actual: [0 1], [49 18], Predictions: [0 1], [55 12]

None


[('hf_rr', 176), ('rmssd', 145), ('sdnn', 140), ('bpm', 128), ('lf_rr', 120)]

One or more of the test scores are non-finite: [0.57255201 0.59406025 0.60945273 0.61327355 0.62014908 0.56460943
 0.58172214 0.58391804 0.5924106  0.60019586 0.57358257 0.5853035
 0.58747729 0.59524329 0.60415909 0.56188191 0.58865565 0.58932781
 0.60498549 0.59938961 0.56303651 0.5896984  0.587295   0.58956401
 0.59270737        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [34 38], Predictions: [0 1], [45 27]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [34 38], Predictions: [0 1], [54 18]
Training RF ...
Model RF, Actual: [0 1], [34 38], Predictions: [0 1], [52 20]
Training XGB ...
Model XGB, Actual: [0 1], [34 38], Predictions: [0 1], [54 18]

None


[('bpm', 116), ('lf_rr', 102), ('rmssd', 101), ('sdnn', 85), ('hf_rr', 64)]


...

Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'sigmoid'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [35 29], Predictions: [0 1], [42 22]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [35 29], Predictions: [0 1], [54 10]
Training RF ...
Model RF, Actual: [0 1], [35 29], Predictions: [0 1], [42 22]
Training XGB ...
Model XGB, Actual: [0 1], [35 29], Predictions: [0 1], [42 22]

None


[('bpm', 109), ('rmssd', 102), ('hf_rr', 98), ('lf_rr', 97), ('sdnn', 89

One or more of the test scores are non-finite: [0.60804599 0.63250084 0.62325002 0.62697019 0.6342079  0.60280522
 0.61535998 0.6058982  0.60943757 0.62011681 0.60674247 0.6160868
 0.60643341 0.6122525  0.61422253 0.59839296 0.59678334 0.59781117
 0.6158534  0.61207408 0.59373143 0.59816984 0.5976544  0.61442716
 0.6151537         nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 30, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [47 26], Predictions: [0 1], [66  7]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [47 26], Predictions: [0 1], [45 28]
Training RF ...
Model RF, Actual: [0 1], [47 26], Predictions: [0 1], [53 20]
Training XGB ...
Model XGB, Actual: [0 1], [47 26], Predictions: [0 1], [60 13]

None


[('sdnn', 291), ('bpm', 289), ('rmssd', 249), ('lf_rr', 219), ('hf_rr', 217)]

One or more of the test scores are non-finite: [0.55390518 0.5616905  0.56391917 0.56245124 0.56683486 0.56487223
 0.57841484 0.57895434 0.59010834 0.58373894 0.6003178  0.5947913
 0.59020297 0.58705326 0.5794595  0.5780285  0.5649273  0.56269289
 0.56167165 0.56856418 0.56768336 0.5731735  0.55803907 0.56142606
 0.56781279        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'sigmoid'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [45 22], Predictions: [0 1], [34 33]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [45 22], Predictions: [0 1], [55 12]
Training RF ...
Model RF, Actual: [0 1], [45 22], Predictions: [0 1], [49 18]
Training XGB ...
Model XGB, Actual: [0 1], [45 22], Predictions: [0 1], [58  9]

None


[('lf_rr', 127), ('bpm', 122), ('sdnn', 121), ('hf_rr', 105), ('rmssd', 

In [8]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split, StratifiedGroupKFold, GroupShuffleSplit

test_size = 0.15
folds = 5

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds)

estimators = [(name, best_models[name]) for name in best_models.keys()]
ensemble = VotingClassifier(estimators, voting="hard")
ensemble.fit(x_train, y_train.loc[:, "label"])
ensemble.score(x_test, y_test.loc[:, "label"])

0.4574468085106383